This notebook imports the GPR model published in the article titled "Machine Learning for Predictive Understanding of Spall Strength of Metals and Alloys" and offers predictions for the spall strength of new candidates. For more detailed information, please refer to
[hyperlink](paper).

In [1]:
#@title Predict the Spall Strength for a Metal or Alloy
#@markdown ***Please first input your parameters, and then press on the left button to run.***

# Import packages
import os
import pickle
import pandas as pd
import sklearn
from sklearn.preprocessing import MinMaxScaler
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF

# Import data
if os.path.exists("/usr/local/spall_strength_prediction"):
  pass
else:
  %cd /usr/local
  !git clone -q https://github.com/Ramprasad-Group/spall_strength_prediction.git
    

data = pd.read_csv("/usr/local/spall_strength_prediction/datasets/metals_and_alloys.csv")

data_X = data.to_numpy()[:,3:]
data_Y = data.to_numpy()[:,2]

### Descriptor scaling ###
scalerM = MinMaxScaler()
scalerM.fit(data_X)
data_X_scaled = scalerM.transform(data_X)


# Build the GPR model
gpr_final = GaussianProcessRegressor(alpha=2,
                                     kernel=3.5**2 * RBF(length_scale=.355)  ,
                                     n_restarts_optimizer = 10,
                                     normalize_y=True,
                                     optimizer=None,
                                     random_state=1
                                    )

GPR = gpr_final.fit(data_X_scaled, data_Y);


/usr/local
fatal: could not create work tree dir 'spall_strength_prediction': Permission denied


/usr/local/lib/python3.11/site-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [2]:
## Making predictions using a form

# @markdown 1. Ultimate Tensile Strength (UTS) (MPa):
UTS = 1110 # @param {type:"number"}
# @markdown 2. Tensile Yield Strength (MPa):
yield_strength = 774 # @param {type:"number"}
# @markdown 3. Bulk Modulus (GPa):
bulk_modulus = 110 # @param {type:"number"}
# @markdown 4. Young's Modulus (GPa):
youngs_modulus = 70 # @param {type:"number"}
# @markdown 5. Density (g/cc):
density = 9.36 # @param {type:"number"}
# @markdown 6. Vickers Hardness (HV):
hardness = 372 # @param {type:"number"}
# @markdown 7. Fracture Toughness (MPa√m):
fracture_toughness = 62 # @param {type:"number"}

# Create a pandas DataFrame
column_names = ['UTS', 'yield_strength', 'bulk_modulus', 'youngs_modulus', 'density',
       'hardness', 'fracture_toughness']
input_data_list = [UTS,yield_strength, bulk_modulus, youngs_modulus, density,
                   hardness, fracture_toughness]
input_data = pd.DataFrame(columns=column_names)
input_data.loc[len(input_data)] = input_data_list

input_data_scaled = scalerM.transform(input_data.values)



#GPR
input_data_GPR, input_data_GPR_std = GPR.predict(input_data_scaled, return_std=True)


print('Predicted Spall Strength using Gaussian Process Regression (GPR) model is: %.3f ± %.3f GPa' %(input_data_GPR[0], input_data_GPR_std[0]))



Predicted Spall Strength using Gaussian Process Regression (GPR) model is: 3.603 ± 3.372 GPa


In [3]:
## Batch prediction

batch_input =pd.read_csv("/Users/keara/Dropbox (GaTech)/spall/for_Github/datasets/validation_metals_and_alloys.csv")
batch_input_X = batch_input.to_numpy()[:,3:]

### Descriptor scaling ###
batch_input_X_scaled = scalerM.transform(batch_input_X)

#GPR
batch_GPR, batch_GPR_std = GPR.predict(batch_input_X_scaled, return_std=True)

for i in range(0, len(batch_input)):
    print('Predicted Spall Strength of %s using Gaussian Process Regression (GPR) model is: %.3f ± %.3f GPa' %(batch_input['name'][i], batch_GPR[i], batch_GPR_std[i]))



Predicted Spall Strength of 93W using Gaussian Process Regression (GPR) model is: 3.086 ± 2.123 GPa
Predicted Spall Strength of HfZrTiTa using Gaussian Process Regression (GPR) model is: 3.603 ± 3.372 GPa
Predicted Spall Strength of Ti 5553 using Gaussian Process Regression (GPR) model is: 2.944 ± 2.021 GPa


# References and further reading

- Keara Frawley, Harikrishna Sahu, Naresh Thadhani, Rampi Ramprasad, *Machine Learning for Predictive Understanding of Spall Strength of Metals and Alloys*, citation info, DOI: [hyperlink](paper)